# Первоначальное решение

## Подготовка датасета

In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("train_data/cntrbtrs_clnts_ops_trn.csv", sep=';', encoding='windows-1251')
output_df = pd.read_csv('train_data/sample_submission.csv') #outputdf
#money_df = pd.read_csv('train_data/trnsctns_ops_trn.csv', sep=';', encoding='windows-1251')

C:\Users\skorp\AppData\Local\Temp\ipykernel_4756\2050464915.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train_data/cntrbtrs_clnts_ops_trn.csv", sep=';', encoding='windows-1251')


In [4]:
df = df.drop(columns=['slctn_nmbr', 'clnt_id', 'accnt_id', 'accnt_bgn_date'])

In [5]:
df = df.drop(columns=['brth_yr', 'brth_plc']) #Возможно верну

In [6]:
df = df.drop(columns=['dstrct', 'city', 'sttlmnt', 'pstl_code', 'addrss_type']) #Оставляем только регион так как от него зависит pnsn_age

In [7]:
df = df.drop(columns=['phn', 'email', 'lk'])

In [8]:
df = df.drop(columns=['okato']) #Выброшен из-за того что слетела кодировка, однако благодаря Okato можно найти данные о городе, регионе и т.д.

In [9]:
df['rgn'] = df['rgn'].fillna('Нет данных')

In [10]:
df = df.fillna(0.0)

In [11]:
df.loc[df['prvs_npf'] != 0.0, 'prvs_npf'] = 1.0 #если человек впервые воспользовался нашими услугами 0 | 1 - если пришел от других компаний

In [25]:
df[:5]

,gndr,prsnt_age,cprtn_prd_d,erly_pnsn_flg,accnt_status,pnsn_age,prvs_npf,rgn,assgn_npo,assgn_ops
0,ж,64,96,0,Накопительный период,55,1.0,ОМСКАЯ ОБЛ,нет,нет
1,м,70,283,0,Накопительный период,60,1.0,АМУРСКАЯ ОБЛ,нет,нет
2,м,69,88,0,Накопительный период,60,1.0,МОСКВА Г,нет,нет
3,ж,62,1301,0,Накопительный период,55,0.0,ТАТАРСТАН РЕСП,нет,нет
4,м,69,106,0,Накопительный период,60,1.0,УДМУРТСКАЯ РЕСП,нет,нет


In [27]:
df.describe()

,prsnt_age,cprtn_prd_d,erly_pnsn_flg,pnsn_age
count,532825.000000,532825.000000,532825.000000,532825.000000
mean,64.142499,324.073215,0.036367,57.073424
std,3.693279,581.011155,0.187201,2.566922
min,37.000000,0.000000,0.000000,55.000000
25%,62.000000,108.000000,0.000000,55.000000
50%,64.000000,182.000000,0.000000,55.000000
75%,66.000000,326.000000,0.000000,60.000000
max,99.000000,7269.000000,1.000000,65.000000


In [29]:
#Балансируем данные (чтобы нейронная сеть/модель машинного обучения не ставила 0 класс и была права в 80% случаях а искала закономерности)
df1 = df.sort_values('erly_pnsn_flg', ascending=False)[:19370]     
df0 = df.sort_values('erly_pnsn_flg', ascending=False)[19370:].sample(frac=0.04)
df_fin = pd.concat([df1, df0], ignore_index=True)
df_fin = df_fin.sample(frac=1)

In [31]:
df_fin

,gndr,prsnt_age,cprtn_prd_d,erly_pnsn_flg,accnt_status,pnsn_age,prvs_npf,rgn,assgn_npo,assgn_ops
16075,ж,57,1586,1,Выплатной период,59,0.0,АРХАНГЕЛЬСКАЯ ОБЛ,нет,нет
16792,ж,53,453,1,Выплатной период,60,1.0,ЧЕЛЯБИНСКАЯ ОБЛ,нет,нет
7421,ж,54,2593,1,Выплатной период,60,1.0,МОСКВА Г,нет,нет
14132,м,55,3173,1,Выплатной период,65,0.0,ЛИПЕЦКАЯ ОБЛ,нет,нет
36890,ж,65,411,0,Накопительный период,55,1.0,ОРЕНБУРГСКАЯ ОБЛ,нет,нет
...,...,...,...,...,...,...,...,...,...,...
20227,ж,69,126,0,Накопительный период,55,1.0,НИЖЕГОРОДСКАЯ ОБЛ,нет,нет
30425,м,64,106,0,Накопительный период,61,1.0,ДАГЕСТАН РЕСП,нет,нет
3545,ж,56,1492,1,Выплатной период,60,1.0,КАРЕЛИЯ РЕСП,нет,нет
2250,ж,57,795,1,Выплатной период,59,1.0,МОСКОВСКАЯ ОБЛ,нет,нет


In [33]:
x = df_fin.drop(columns=['erly_pnsn_flg'])
y = df_fin['erly_pnsn_flg']

In [35]:
print(f"0 = {y.tolist().count(0)}, 1 = {y.tolist().count(1)}")

0 = 20538, 1 = 19370


In [37]:
x['gndr'] = x['gndr'].replace(['ж', 'м'], [0.0, 1.0]) # 0 - жен | 1 - муж

C:\Users\skorp\AppData\Local\Temp\ipykernel_4756\4178529825.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['gndr'] = x['gndr'].replace(['ж', 'м'], [0.0, 1.0]) # 0 - жен | 1 - муж


In [39]:
x['accnt_status'] = x['accnt_status'].replace(['Выплатной период', 'Накопительный период'], [0.0, 1.0]) # 1 - накопительный период | 0 - Выплатной период
x['assgn_npo'] = x['assgn_npo'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору НПО | 1 - является
x['assgn_ops'] = x['assgn_ops'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору ОПС | 1 - является

C:\Users\skorp\AppData\Local\Temp\ipykernel_4756\615035629.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['accnt_status'] = x['accnt_status'].replace(['Выплатной период', 'Накопительный период'], [0.0, 1.0]) # 1 - накопительный период | 0 - Выплатной период
C:\Users\skorp\AppData\Local\Temp\ipykernel_4756\615035629.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['assgn_npo'] = x['assgn_npo'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору НПО | 1 - является
C:\Users\skorp\AppData\Local

In [41]:
x

,gndr,prsnt_age,cprtn_prd_d,accnt_status,pnsn_age,prvs_npf,rgn,assgn_npo,assgn_ops
16075,0.0,57,1586,0.0,59,0.0,АРХАНГЕЛЬСКАЯ ОБЛ,0.0,0.0
16792,0.0,53,453,0.0,60,1.0,ЧЕЛЯБИНСКАЯ ОБЛ,0.0,0.0
7421,0.0,54,2593,0.0,60,1.0,МОСКВА Г,0.0,0.0
14132,1.0,55,3173,0.0,65,0.0,ЛИПЕЦКАЯ ОБЛ,0.0,0.0
36890,0.0,65,411,1.0,55,1.0,ОРЕНБУРГСКАЯ ОБЛ,0.0,0.0
...,...,...,...,...,...,...,...,...,...
20227,0.0,69,126,1.0,55,1.0,НИЖЕГОРОДСКАЯ ОБЛ,0.0,0.0
30425,1.0,64,106,1.0,61,1.0,ДАГЕСТАН РЕСП,0.0,0.0
3545,0.0,56,1492,0.0,60,1.0,КАРЕЛИЯ РЕСП,0.0,0.0
2250,0.0,57,795,0.0,59,1.0,МОСКОВСКАЯ ОБЛ,0.0,0.0


### Нормализация данных

In [44]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

In [45]:
x_test = x.copy() #Создаем копию над которой будем тестировать скейлеры

In [48]:
prsnt_age_scaler = MinMaxScaler()
pnsn_age_scaler = MinMaxScaler()
cprtn_scaler = StandardScaler()
rgn_encoder = LabelEncoder()
rgn_scaler = MinMaxScaler()

In [50]:
prsnt_x = x.iloc[:, 1:2] #Для трансформации скейлера
prsnt_df = df.iloc[:, 1:2] #Для обучения скейлера

pnsn_x = x.iloc[:, 4:5]
pnsn_df = df.iloc[:, 5:6]

cprtn_x = x.iloc[:, 2:3]
cprtn_df = df.iloc[:, 2:3]

sup_df = df.iloc[:, 7:8].copy()

In [52]:
prsnt_age_scaler.fit(prsnt_df)
pnsn_age_scaler.fit(pnsn_df)
cprtn_scaler.fit(cprtn_df)
rgn_encoder.fit(df['rgn'])
sup_df['rgn'] = rgn_encoder.transform(df['rgn'])
rgn_scaler.fit(sup_df)

MinMaxScaler()

In [54]:
x_test['prsnt_age'] = prsnt_age_scaler.transform(prsnt_x)
x_test['pnsn_age'] = pnsn_age_scaler.transform(pnsn_x)
x_test['cprtn_prd_d'] = cprtn_scaler.transform(cprtn_x)
x_test['rgn'] = rgn_encoder.transform(x['rgn'])
x_test['rgn'] = rgn_scaler.transform(x_test.iloc[:, 6:7])
x = x_test.copy()

### Выгрузка датасетов и скейлеров

In [57]:
import pickle
x.to_csv('datasets/x.csv', index=False)
y.to_csv('datasets/y.csv', index=False)
pickle.dump(prsnt_age_scaler, open('scalers/prsnt_age_scaler.sav', 'wb'))
pickle.dump(pnsn_age_scaler, open('scalers/pnsn_age_scaler.sav', 'wb'))
pickle.dump(cprtn_scaler, open('scalers/cprtn_scaler.sav', 'wb'))
pickle.dump(rgn_encoder, open('scalers/rgn_encoder.sav', 'wb'))
pickle.dump(rgn_scaler, open('scalers/rgn_scaler.sav', 'wb'))

## Обучение модели

In [60]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.33, random_state=42)

In [90]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [101]:
model = Sequential()
model.add(Dense(units = 400 , activation = 'relu' , input_dim = X_train.shape[-1]))
model.add(Dense(units = 200 , activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(units = 100 , activation = 'relu'))
model.add(Dense(units = 50 , activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(units = 4 , activation = 'relu'))
model.add(Dense(units = 1 , activation = 'sigmoid'))
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['binary_accuracy', 'f1_score'])
model.summary()

G:\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_27 (Dense)                     │ (None, 400)                 │           4,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 200)                 │          80,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 200)                 │             800 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 100)                 │          20,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 50)                  │             200 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 4)                   │             204 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_32 (Dense)                     │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 110,559 (431.87 KB)

 Trainable params: 110,059 (429.92 KB)

 Non-trainable params: 500 (1.95 KB)

In [103]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.3)

Epoch 1/100
585/585 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - binary_accuracy: 0.9824 - f1_score: 0.6598 - loss: 0.0720 - val_binary_accuracy: 0.9978 - val_f1_score: 0.6433 - val_loss: 0.0113
Epoch 2/100
585/585 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - binary_accuracy: 0.9979 - f1_score: 0.6642 - loss: 0.0114 - val_binary_accuracy: 0.9985 - val_f1_score: 0.6433 - val_loss: 0.0061
Epoch 3/100
585/585 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - binary_accuracy: 0.9978 - f1_score: 0.6644 - loss: 0.0102 - val_binary_accuracy: 0.9978 - val_f1_score: 0.6433 - val_loss: 0.0079
Epoch 4/100
585/585 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - binary_accuracy: 0.9974 - f1_score: 0.6623 - loss: 0.0091 - val_binary_accuracy: 0.9979 - val_f1_score: 0.6433 - val_loss: 0.0038
Epoch 5/100
585/585 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - binary_accuracy: 0.9972 - f1_score: 0.6696 - loss: 0.0075 - val_binary_accuracy: 0.9978 - val_f1_score: 0.6433 - val_loss: 0.0112
Epoch 6/100
585/585 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - binary_accuracy: 0.9

In [105]:
model.evaluate(X_test, y_test)

412/412 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - binary_accuracy: 0.9972 - f1_score: 0.9979 - loss: 0.0037


[0.003424415597692132, 0.9979484677314758, 0.9970789551734924]

## Тестирование модели

In [109]:
df_test = df.copy()

In [111]:
df_test

,gndr,prsnt_age,cprtn_prd_d,erly_pnsn_flg,accnt_status,pnsn_age,prvs_npf,rgn,assgn_npo,assgn_ops
0,ж,64,96,0,Накопительный период,55,1.0,ОМСКАЯ ОБЛ,нет,нет
1,м,70,283,0,Накопительный период,60,1.0,АМУРСКАЯ ОБЛ,нет,нет
2,м,69,88,0,Накопительный период,60,1.0,МОСКВА Г,нет,нет
3,ж,62,1301,0,Накопительный период,55,0.0,ТАТАРСТАН РЕСП,нет,нет
4,м,69,106,0,Накопительный период,60,1.0,УДМУРТСКАЯ РЕСП,нет,нет
...,...,...,...,...,...,...,...,...,...,...
532820,ж,59,490,0,Накопительный период,56,1.0,НИЖЕГОРОДСКАЯ ОБЛ,нет,нет
532821,м,68,307,0,Накопительный период,60,1.0,КАРЕЛИЯ РЕСП,нет,нет
532822,м,65,97,0,Накопительный период,60,1.0,БАШКОРТОСТАН РЕСП,нет,нет
532823,ж,65,139,0,Накопительный период,55,1.0,ТАМБОВСКАЯ ОБЛ,нет,нет


In [113]:
x = df_test.drop(columns=['erly_pnsn_flg'])
y = df_test['erly_pnsn_flg']

In [115]:
print(f"0 = {y.tolist().count(0)}, 1 = {y.tolist().count(1)}")

0 = 512763, 1 = 19370


In [117]:
x['gndr'] = x['gndr'].replace(['ж', 'м'], [0.0, 1.0]) # 0 - жен | 1 - муж

C:\Users\skorp\AppData\Local\Temp\ipykernel_11712\4178529825.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['gndr'] = x['gndr'].replace(['ж', 'м'], [0.0, 1.0]) # 0 - жен | 1 - муж


In [119]:
x['accnt_status'] = x['accnt_status'].replace(['Выплатной период', 'Накопительный период'], [0.0, 1.0]) # 1 - накопительный период | 0 - Выплатной период
x['assgn_npo'] = x['assgn_npo'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору НПО | 1 - является
x['assgn_ops'] = x['assgn_ops'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору ОПС | 1 - является

C:\Users\skorp\AppData\Local\Temp\ipykernel_11712\615035629.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['accnt_status'] = x['accnt_status'].replace(['Выплатной период', 'Накопительный период'], [0.0, 1.0]) # 1 - накопительный период | 0 - Выплатной период
C:\Users\skorp\AppData\Local\Temp\ipykernel_11712\615035629.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['assgn_npo'] = x['assgn_npo'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору НПО | 1 - является
C:\Users\skorp\AppData\Loc

In [121]:
x

,gndr,prsnt_age,cprtn_prd_d,accnt_status,pnsn_age,prvs_npf,rgn,assgn_npo,assgn_ops
0,0.0,64,96,1.0,55,1.0,ОМСКАЯ ОБЛ,0.0,0.0
1,1.0,70,283,1.0,60,1.0,АМУРСКАЯ ОБЛ,0.0,0.0
2,1.0,69,88,1.0,60,1.0,МОСКВА Г,0.0,0.0
3,0.0,62,1301,1.0,55,0.0,ТАТАРСТАН РЕСП,0.0,0.0
4,1.0,69,106,1.0,60,1.0,УДМУРТСКАЯ РЕСП,0.0,0.0
...,...,...,...,...,...,...,...,...,...
532820,0.0,59,490,1.0,56,1.0,НИЖЕГОРОДСКАЯ ОБЛ,0.0,0.0
532821,1.0,68,307,1.0,60,1.0,КАРЕЛИЯ РЕСП,0.0,0.0
532822,1.0,65,97,1.0,60,1.0,БАШКОРТОСТАН РЕСП,0.0,0.0
532823,0.0,65,139,1.0,55,1.0,ТАМБОВСКАЯ ОБЛ,0.0,0.0


### Нормализация данных

In [124]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

In [126]:
x_test = x.copy() #Создаем копию над которой будем тестировать скейлеры

In [128]:
prsnt_x = x.iloc[:, 1:2] #Для трансформации скейлера
prsnt_df = df.iloc[:, 1:2] #Для обучения скейлера

pnsn_x = x.iloc[:, 4:5]
pnsn_df = df.iloc[:, 5:6]

cprtn_x = x.iloc[:, 2:3]
cprtn_df = df.iloc[:, 2:3]

sup_df = df.iloc[:, 7:8].copy()

In [130]:
x_test['prsnt_age'] = prsnt_age_scaler.transform(prsnt_x)
x_test['pnsn_age'] = pnsn_age_scaler.transform(pnsn_x)
x_test['cprtn_prd_d'] = cprtn_scaler.transform(cprtn_x)
x_test['rgn'] = rgn_encoder.transform(x['rgn'])
x_test['rgn'] = rgn_scaler.transform(x_test.iloc[:, 6:7])
x = x_test.copy()

In [132]:
x

,gndr,prsnt_age,cprtn_prd_d,accnt_status,pnsn_age,prvs_npf,rgn,assgn_npo,assgn_ops
0,0.0,0.435484,-0.392589,1.0,0.0,1.0,0.564356,0.0,0.0
1,1.0,0.532258,-0.070810,1.0,0.5,1.0,0.029703,0.0,0.0
2,1.0,0.516129,-0.406355,1.0,0.5,1.0,0.485149,0.0,0.0
3,0.0,0.403226,1.680909,1.0,0.0,0.0,0.792079,0.0,0.0
4,1.0,0.516129,-0.375382,1.0,0.5,1.0,0.851485,0.0,0.0
...,...,...,...,...,...,...,...,...,...
532820,0.0,0.354839,0.285384,1.0,0.1,1.0,0.534653,0.0,0.0
532821,1.0,0.500000,-0.029512,1.0,0.5,1.0,0.316832,0.0,0.0
532822,1.0,0.451613,-0.390869,1.0,0.5,1.0,0.069307,0.0,0.0
532823,0.0,0.451613,-0.318597,1.0,0.0,1.0,0.782178,0.0,0.0


In [134]:
model.evaluate(x, y)

16630/16630 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - binary_accuracy: 0.9996 - f1_score: 0.9539 - loss: 0.0027


[0.002729990053921938, 0.9996278882026672, 0.9527087807655334]

### Выгрузка модели

In [13]:
model.save('models/Pensia_classifier.keras')
model.save('models/Pensia_classifier.h5')

## MVP

Загрузка датасета для тестирования, скейлеров и модели

In [1]:
import pickle
import pandas as pd
prsnt_age_scaler = pickle.load(open('scalers/prsnt_age_scaler.sav', 'rb'))
pnsn_age_scaler = pickle.load(open('scalers/pnsn_age_scaler.sav', 'rb'))
cprtn_scaler = pickle.load(open('scalers/cprtn_scaler.sav', 'rb'))
rgn_encoder = pickle.load(open('scalers/rgn_encoder.sav', 'rb'))
rgn_scaler = pickle.load(open('scalers/rgn_scaler.sav', 'rb'))
df = pd.read_csv("train_data/cntrbtrs_clnts_ops_trn.csv", sep=';', encoding='windows-1251')

C:\Users\skorp\AppData\Local\Temp\ipykernel_6028\917607231.py:8: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train_data/cntrbtrs_clnts_ops_trn.csv", sep=';', encoding='windows-1251')


In [2]:
#Загрузка модели
from tensorflow.keras.models import load_model
model = load_model('models/Pensia_classifier.keras', compile=False)
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['binary_accuracy', 'f1_score'])

In [3]:
finaly_df = df.iloc[:, 2:3].copy()

In [4]:
df = df.drop(columns=['slctn_nmbr',
                      'clnt_id',
                      'accnt_id',
                      'accnt_bgn_date',
                      'brth_yr',
                      'brth_plc',
                      'dstrct',
                      'city',
                      'sttlmnt',
                      'pstl_code',
                      'addrss_type',
                      'phn',
                      'email',
                      'lk',
                      'okato'])

In [5]:
df['rgn'] = df['rgn'].fillna('Нет данных')

In [6]:
df = df.fillna(0.0)

In [7]:
df.loc[df['prvs_npf'] != 0.0, 'prvs_npf'] = 1.0 #если человек впервые воспользовался нашими услугами 0 | 1 - если пришел от других компаний

In [8]:
x = df.drop(columns=['erly_pnsn_flg'])
y = df['erly_pnsn_flg']

In [9]:
x['gndr'] = x['gndr'].replace(['ж', 'м'], [0.0, 1.0]) # 0 - жен | 1 - муж

C:\Users\skorp\AppData\Local\Temp\ipykernel_6028\4178529825.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['gndr'] = x['gndr'].replace(['ж', 'м'], [0.0, 1.0]) # 0 - жен | 1 - муж


In [10]:
x['accnt_status'] = x['accnt_status'].replace(['Выплатной период', 'Накопительный период'], [0.0, 1.0]) # 1 - накопительный период | 0 - Выплатной период
x['assgn_npo'] = x['assgn_npo'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору НПО | 1 - является
x['assgn_ops'] = x['assgn_ops'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору ОПС | 1 - является

C:\Users\skorp\AppData\Local\Temp\ipykernel_6028\615035629.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['accnt_status'] = x['accnt_status'].replace(['Выплатной период', 'Накопительный период'], [0.0, 1.0]) # 1 - накопительный период | 0 - Выплатной период
C:\Users\skorp\AppData\Local\Temp\ipykernel_6028\615035629.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['assgn_npo'] = x['assgn_npo'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору НПО | 1 - является
C:\Users\skorp\AppData\Local

### Нормализация данных

In [12]:
prsnt_x = x.iloc[:, 1:2] 
pnsn_x = x.iloc[:, 4:5]
cprtn_x = x.iloc[:, 2:3]

In [14]:
x['prsnt_age'] = prsnt_age_scaler.transform(prsnt_x)
x['pnsn_age'] = pnsn_age_scaler.transform(pnsn_x)
x['cprtn_prd_d'] = cprtn_scaler.transform(cprtn_x)
x['rgn'] = rgn_encoder.transform(x['rgn'])
x['rgn'] = rgn_scaler.transform(x.iloc[:, 6:7])

In [15]:
x_pred = model.predict(x)

16651/16651 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step


In [16]:
erly_pnsn_flg = []
for x in x_pred:
    if x >= 0.9:
        erly_pnsn_flg.append(1)
    else:
        erly_pnsn_flg.append(0)

In [17]:
finaly_df['erly_pnsn_flg'] = erly_pnsn_flg
finaly_df[:5]

,accnt_id,erly_pnsn_flg
0,0xFFB14DE2D28AAD45B08AC54ADDFE6AD2,0
1,0x222F5E5BA3715F418662A3C65B0B051F,0
2,0x943D2C768A4FB38311E659ACF7E960C5,0
3,0xB7BAA18DCFE2E842A7AF84BBE185265C,0
4,0x68D14914CF3A79408F6A6E097C1A61AF,0


In [18]:
finaly_df['erly_pnsn_flg'].tolist().count(1)

19316

In [19]:
y.tolist().count(1)

19377

In [36]:
finaly_df.to_csv('datasets/submission.csv', index=False)

# Решение без текущего возраста и Статуса Аккаунта

## Подготовка датасета

In [510]:
import pandas as pd
import numpy as np

In [512]:
df = pd.read_csv("train_data/cntrbtrs_clnts_ops_trn.csv", sep=';', encoding='windows-1251')
output_df = pd.read_csv('train_data/sample_submission.csv') #outputdf
#money_df = pd.read_csv('train_data/trnsctns_ops_trn.csv', sep=';', encoding='windows-1251')

C:\Users\skorp\AppData\Local\Temp\ipykernel_12828\2050464915.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train_data/cntrbtrs_clnts_ops_trn.csv", sep=';', encoding='windows-1251')


In [513]:
df = df.drop(columns=['slctn_nmbr', 'clnt_id', 'accnt_id', 'accnt_bgn_date'])

In [514]:
df = df.drop(columns=['brth_yr', 'brth_plc']) #Возможно верну

In [515]:
df = df.drop(columns=['dstrct', 'city', 'sttlmnt', 'pstl_code', 'addrss_type']) #Оставляем только регион так как от него зависит pnsn_age

In [516]:
df = df.drop(columns=['phn', 'email', 'lk'])

In [517]:
df = df.drop(columns=['okato']) #Выброшен из-за того что слетела кодировка, однако благодаря Okato можно найти данные о городе, регионе и т.д.

In [518]:
df = df.drop(columns=['prsnt_age', 'accnt_status'])

In [519]:
df['rgn'] = df['rgn'].fillna('Нет данных')

In [520]:
df = df.fillna(0.0)

In [521]:
df.loc[df['prvs_npf'] != 0.0, 'prvs_npf'] = 1.0 #если человек впервые воспользовался нашими услугами 0 | 1 - если пришел от других компаний

In [522]:
df[:5]

,gndr,cprtn_prd_d,erly_pnsn_flg,pnsn_age,prvs_npf,rgn,assgn_npo,assgn_ops
0,ж,96,0,55,1.0,ОМСКАЯ ОБЛ,нет,нет
1,м,283,0,60,1.0,АМУРСКАЯ ОБЛ,нет,нет
2,м,88,0,60,1.0,МОСКВА Г,нет,нет
3,ж,1301,0,55,0.0,ТАТАРСТАН РЕСП,нет,нет
4,м,106,0,60,1.0,УДМУРТСКАЯ РЕСП,нет,нет


In [523]:
df.describe()

,cprtn_prd_d,erly_pnsn_flg,pnsn_age
count,532825.000000,532825.000000,532825.000000
mean,324.073215,0.036367,57.073424
std,581.011155,0.187201,2.566922
min,0.000000,0.000000,55.000000
25%,108.000000,0.000000,55.000000
50%,182.000000,0.000000,55.000000
75%,326.000000,0.000000,60.000000
max,7269.000000,1.000000,65.000000


In [524]:
#Балансируем данные (чтобы нейронная сеть/модель машинного обучения не ставила 0 класс и была права в 80% случаях а искала закономерности)
df1 = df.sort_values('erly_pnsn_flg', ascending=False)[:19370]     
df0 = df.sort_values('erly_pnsn_flg', ascending=False)[19370:].sample(frac=0.04)
df_fin = pd.concat([df1, df0], ignore_index=True)
df_fin = df_fin.sample(frac=1)

In [525]:
df_fin

,gndr,cprtn_prd_d,erly_pnsn_flg,pnsn_age,prvs_npf,rgn,assgn_npo,assgn_ops
14808,ж,3407,1,60,1.0,САРАТОВСКАЯ ОБЛ,нет,нет
17061,ж,3224,1,59,0.0,КРАСНОДАРСКИЙ КРАЙ,нет,нет
15650,ж,5495,1,60,0.0,САНКТ-ПЕТЕРБУРГ Г,нет,нет
21074,ж,153,0,55,1.0,РЯЗАНСКАЯ ОБЛ,нет,нет
38009,м,185,0,60,0.0,ОМСКАЯ ОБЛ,нет,нет
...,...,...,...,...,...,...,...,...
6047,ж,1052,1,60,1.0,КРАСНОЯРСКИЙ КРАЙ,нет,нет
22604,м,92,0,60,1.0,ОРЕНБУРГСКАЯ ОБЛ,нет,нет
30886,ж,157,0,55,1.0,ВОЛГОГРАДСКАЯ ОБЛ,нет,нет
29829,м,109,0,60,1.0,ЧЕЛЯБИНСКАЯ ОБЛ,нет,нет


In [526]:
x = df_fin.drop(columns=['erly_pnsn_flg'])
y = df_fin['erly_pnsn_flg']

In [531]:
print(f"0 = {y.tolist().count(0)}, 1 = {y.tolist().count(1)}")

0 = 20538, 1 = 19370


In [539]:
x['gndr'] = x['gndr'].replace(['ж', 'м'], [0.0, 1.0]) # 0 - жен | 1 - муж

C:\Users\skorp\AppData\Local\Temp\ipykernel_12828\4178529825.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['gndr'] = x['gndr'].replace(['ж', 'м'], [0.0, 1.0]) # 0 - жен | 1 - муж


In [546]:
x['assgn_npo'] = x['assgn_npo'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору НПО | 1 - является
x['assgn_ops'] = x['assgn_ops'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору ОПС | 1 - является

C:\Users\skorp\AppData\Local\Temp\ipykernel_12828\1879176768.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['assgn_npo'] = x['assgn_npo'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору НПО | 1 - является
C:\Users\skorp\AppData\Local\Temp\ipykernel_12828\1879176768.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['assgn_ops'] = x['assgn_ops'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору ОПС | 1 - является


In [548]:
x

,gndr,cprtn_prd_d,pnsn_age,prvs_npf,rgn,assgn_npo,assgn_ops
14808,0.0,3407,60,1.0,САРАТОВСКАЯ ОБЛ,0.0,0.0
17061,0.0,3224,59,0.0,КРАСНОДАРСКИЙ КРАЙ,0.0,0.0
15650,0.0,5495,60,0.0,САНКТ-ПЕТЕРБУРГ Г,0.0,0.0
21074,0.0,153,55,1.0,РЯЗАНСКАЯ ОБЛ,0.0,0.0
38009,1.0,185,60,0.0,ОМСКАЯ ОБЛ,0.0,0.0
...,...,...,...,...,...,...,...
6047,0.0,1052,60,1.0,КРАСНОЯРСКИЙ КРАЙ,0.0,0.0
22604,1.0,92,60,1.0,ОРЕНБУРГСКАЯ ОБЛ,0.0,0.0
30886,0.0,157,55,1.0,ВОЛГОГРАДСКАЯ ОБЛ,0.0,0.0
29829,1.0,109,60,1.0,ЧЕЛЯБИНСКАЯ ОБЛ,0.0,0.0


### Нормализация данных

In [551]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

In [553]:
x_test = x.copy() #Создаем копию над которой будем тестировать скейлеры

In [555]:
pnsn_age_scaler = MinMaxScaler()
cprtn_scaler = StandardScaler()
rgn_encoder = LabelEncoder()
rgn_scaler = MinMaxScaler()

In [557]:
pnsn_x = x.iloc[:, 2:3]
pnsn_df = df.iloc[:, 3:4]

cprtn_x = x.iloc[:, 1:2]
cprtn_df = df.iloc[:, 1:2]

sup_df = df.iloc[:, 5:6].copy()

In [559]:
pnsn_age_scaler.fit(pnsn_df)
cprtn_scaler.fit(cprtn_df)
rgn_encoder.fit(df['rgn'])
sup_df['rgn'] = rgn_encoder.transform(df['rgn'])
rgn_scaler.fit(sup_df)

MinMaxScaler()

In [561]:
x_test['pnsn_age'] = pnsn_age_scaler.transform(pnsn_x)
x_test['cprtn_prd_d'] = cprtn_scaler.transform(cprtn_x)
x_test['rgn'] = rgn_encoder.transform(x['rgn'])
x_test['rgn'] = rgn_scaler.transform(x_test.iloc[:, 4:5])
x = x_test.copy()

In [563]:
x_test

,gndr,cprtn_prd_d,pnsn_age,prvs_npf,rgn,assgn_npo,assgn_ops
14808,0.0,5.306145,0.5,1.0,0.696078,0.0,0.0
17061,0.0,4.991177,0.4,0.0,0.382353,0.0,0.0
15650,0.0,8.899884,0.5,0.0,0.686275,0.0,0.0
21074,0.0,-0.294441,0.0,1.0,0.666667,0.0,0.0
38009,1.0,-0.239364,0.5,0.0,0.568627,0.0,0.0
...,...,...,...,...,...,...,...
6047,0.0,1.252863,0.5,1.0,0.392157,0.0,0.0
22604,1.0,-0.399430,0.5,1.0,0.578431,0.0,0.0
30886,0.0,-0.287556,0.0,1.0,0.127451,0.0,0.0
29829,1.0,-0.370171,0.5,1.0,0.931373,0.0,0.0


## Выгрузка датасетов

In [566]:
import pickle
x.to_csv('datasets/x(WAA).csv', index=False)
y.to_csv('datasets/y(WAA).csv', index=False)
pickle.dump(pnsn_age_scaler, open('scalers/pnsn_age_scaler(WAA).sav', 'wb'))
pickle.dump(cprtn_scaler, open('scalers/cprtn_scaler(WAA).sav', 'wb'))
pickle.dump(rgn_encoder, open('scalers/rgn_encoder(WAA).sav', 'wb'))
pickle.dump(rgn_scaler, open('scalers/rgn_scaler(WAA).sav', 'wb'))

## Обучение модели

In [569]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.33, random_state=42)

In [571]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [573]:
model = Sequential()
model.add(Dense(units = 400 , activation = 'relu' , input_dim = X_train.shape[-1]))
model.add(Dense(units = 200 , activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(units = 100 , activation = 'relu'))
model.add(Dense(units = 50 , activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(units = 4 , activation = 'relu'))
model.add(Dense(units = 1 , activation = 'sigmoid'))
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['binary_accuracy', 'f1_score'])
model.summary()

G:\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                     │ (None, 400)                 │           3,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 200)                 │          80,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 200)                 │             800 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 100)                 │          20,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 50)                  │             200 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 4)                   │             204 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 109,759 (428.75 KB)

 Trainable params: 109,259 (426.79 KB)

 Non-trainable params: 500 (1.95 KB)

In [575]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.3)

Epoch 1/100
585/585 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - f1_score: 0.6499 - loss: 0.1185 - val_f1_score: 0.6538 - val_loss: 0.0120
Epoch 2/100
585/585 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - f1_score: 0.6516 - loss: 0.0202 - val_f1_score: 0.6538 - val_loss: 0.0145
Epoch 3/100
585/585 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - f1_score: 0.6546 - loss: 0.0121 - val_f1_score: 0.6538 - val_loss: 0.0062
Epoch 4/100
585/585 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - f1_score: 0.6526 - loss: 0.0099 - val_f1_score: 0.6538 - val_loss: 0.0051
Epoch 5/100
585/585 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - f1_score: 0.6485 - loss: 0.0105 - val_f1_score: 0.6538 - val_loss: 0.0060
Epoch 6/100
585/585 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - f1_score: 0.6456 - loss: 0.0122 - val_f1_score: 0.6538 - val_loss: 0.0043
Epoch 7/100
585/585 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - f1_score: 0.6536 - loss: 0.0100 - val_f1_score: 0.6538 - val_loss: 0.0045
Epoch 8/100
585/585 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - f1_score: 0.6445 - loss: 0.0081 - val_f1_s

Exception ignored in: <function ThreadPoolExecutor._adjust_thread_count.<locals>.weakref_cb at 0x0000021D15F53880>
Traceback (most recent call last):
  File "G:\Anaconda\Lib\concurrent\futures\thread.py", line 191, in weakref_cb
KeyboardInterrupt: 

KeyboardInterrupt



In [447]:
model.evaluate(x, y)

16651/16651 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - binary_accuracy: 0.9966 - f1_score: 0.1003 - loss: 0.0036


[0.0036877067759633064, 0.9965673685073853, 0.0997464656829834]

In [453]:
model.save('models/Pensia_classifier(WAA1).keras')
model.save('models/Pensia_classifier(WAA1).h5')

## MVP

Загрузка датасета для тестирования, скейлеров и модели

In [455]:
import pickle
import pandas as pd
pnsn_age_scaler = pickle.load(open('scalers/pnsn_age_scaler(WAA).sav', 'rb'))
cprtn_scaler = pickle.load(open('scalers/cprtn_scaler(WAA).sav', 'rb'))
rgn_encoder = pickle.load(open('scalers/rgn_encoder(WAA).sav', 'rb'))
rgn_scaler = pickle.load(open('scalers/rgn_scaler(WAA).sav', 'rb'))
df = pd.read_csv("train_data/cntrbtrs_clnts_ops_trn.csv", sep=';', encoding='windows-1251')

C:\Users\skorp\AppData\Local\Temp\ipykernel_12828\1602399030.py:7: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train_data/cntrbtrs_clnts_ops_trn.csv", sep=';', encoding='windows-1251')


In [459]:
#Загрузка модели
from tensorflow.keras.models import load_model
model = load_model('models/Pensia_classifier(WAA1).keras', compile=False)
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['binary_accuracy', 'f1_score'])

In [461]:
finaly_df = df.iloc[:, 2:3].copy()

In [465]:
df = df.drop(columns=['slctn_nmbr',
                      'clnt_id',
                      'accnt_id',
                      'accnt_bgn_date',
                      'brth_yr',
                      'brth_plc',
                      'dstrct',
                      'city',
                      'sttlmnt',
                      'pstl_code',
                      'addrss_type',
                      'phn',
                      'email',
                      'lk',
                      'accnt_status',
                      'prsnt_age',
                      'okato'])

In [467]:
df['rgn'] = df['rgn'].fillna('Нет данных')

In [469]:
df = df.fillna(0.0)

In [471]:
df.loc[df['prvs_npf'] != 0.0, 'prvs_npf'] = 1.0 #если человек впервые воспользовался нашими услугами 0 | 1 - если пришел от других компаний

In [473]:
x = df.drop(columns=['erly_pnsn_flg'])
y = df['erly_pnsn_flg']

In [475]:
x['gndr'] = x['gndr'].replace(['ж', 'м'], [0.0, 1.0]) # 0 - жен | 1 - муж

C:\Users\skorp\AppData\Local\Temp\ipykernel_12828\4178529825.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['gndr'] = x['gndr'].replace(['ж', 'м'], [0.0, 1.0]) # 0 - жен | 1 - муж


In [477]:
x['assgn_npo'] = x['assgn_npo'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору НПО | 1 - является
x['assgn_ops'] = x['assgn_ops'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору ОПС | 1 - является

C:\Users\skorp\AppData\Local\Temp\ipykernel_12828\1879176768.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['assgn_npo'] = x['assgn_npo'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору НПО | 1 - является
C:\Users\skorp\AppData\Local\Temp\ipykernel_12828\1879176768.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['assgn_ops'] = x['assgn_ops'].replace(['нет', 'да'], [0.0, 1.0]) # 0 - не является правопреемником по договору ОПС | 1 - является


### Нормализация данных

In [496]:
pnsn_x = x.iloc[:, 2:3]
cprtn_x = x.iloc[:, 1:2]

In [498]:
x['pnsn_age'] = pnsn_age_scaler.transform(pnsn_x)
x['cprtn_prd_d'] = cprtn_scaler.transform(cprtn_x)
x['rgn'] = rgn_encoder.transform(x['rgn'])
x['rgn'] = rgn_scaler.transform(x.iloc[:, 4:5])

In [500]:
x_pred = model.predict(x)

16651/16651 ━━━━━━━━━━━━━━━━━━━━ 25s 2ms/step


In [501]:
erly_pnsn_flg = []
for x in x_pred:
    if x >= 0.9:
        erly_pnsn_flg.append(1)
    else:
        erly_pnsn_flg.append(0)

In [502]:
finaly_df['erly_pnsn_flg'] = erly_pnsn_flg
finaly_df[:5]

,accnt_id,erly_pnsn_flg
0,0xFFB14DE2D28AAD45B08AC54ADDFE6AD2,0
1,0x222F5E5BA3715F418662A3C65B0B051F,0
2,0x943D2C768A4FB38311E659ACF7E960C5,0
3,0xB7BAA18DCFE2E842A7AF84BBE185265C,0
4,0x68D14914CF3A79408F6A6E097C1A61AF,0


In [503]:
finaly_df['erly_pnsn_flg'].tolist().count(1)

19368

In [504]:
y.tolist().count(1)

19377

In [577]:
finaly_df.to_csv('datasets/submission(AWW).csv', index=False)

# Работа со вторым датасетом